In [1]:
import EDS as eds

required_para = eds.required_parameters(    
    spray_number = [1, 2, 3],
    spray_moment = [30, 45, 60],
    spray_eff = [0.5, 0.5, 0.5],
    
    # for ["Susceptible", "Moderate", "Resistant"]
    p_opt = [7, 10, 14],
    rc_opt_par = [0.35, 0.25, 0.15],
    rrlex_par = [0.1, 0.01, .0001]
)

result = eds.calculate_disease_severity(    
    data_path = "Original_Code/Data.csv",
    info_path = "Original_Code/Info.csv",
    number_of_repeat_year = 4,       
    number_applications_list = [0],
    genetic_mechanistic_list = ["Susceptible"],    
    required_para = required_para,
    output_columns = ["SevMAX"],   
    path_result = None
)

# result.to_csv(f"results.csv", index=None)

result

/home/pooya/miniconda3/lib/python3.9/site-packages/EDS/location_data.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["info_id"] = row[1]["ID"] + "_" + str(row[1]["year"]) + "_" + str(row[1]["planting_date"]) + "_" + str(row[1]["obs_planting_delta"]) + "_" + row[1]["Crop"]
/home/pooya/miniconda3/lib/python3.9/site-packages/EDS/location_data.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["year"] = row[1]["year"]
/home/pooya/miniconda3/lib/python3.9/site-packages/EDS/location_data.py:44: Setti

,locationId,Date1,Date2,N_Days,latitude,longitude,number_applications,genetic_mechanistic,crop,SevMAX
0,ID_44.26405373_-92.9375238_2017_5/3/2017_129_Soy,20170503,20170909,129,44.264054,-92.937524,0,Susceptible,Soy,8.641549e-01
1,ID_44.740789_-91.2186815_2016_5/15/2016_73_Corn,20160515,20160727,73,44.740789,-91.218682,0,Susceptible,Corn,3.540606e-07
2,ID_44.97603894_-95.21102976_2017_5/5/2017_98_Corn,20170505,20170811,98,44.976039,-95.211030,0,Susceptible,Corn,6.978343e-01
3,ID_44.97603894_-95.21102976_2019_5/5/2019_89_Corn,20190505,20190802,89,44.976039,-95.211030,0,Susceptible,Corn,3.766721e-01


# Step to Step Run

In [ ]:
import datetime
import itertools
import math
import numpy as np
import pandas as pd

from Create_Package_Original.EDS.required_parameters import *
from Create_Package_Original.EDS.calculate_disease_severity import *



required_para = required_parameters(    
    spray_number = [1, 2, 3],
    spray_moment = [30, 45, 60],
    spray_eff = [0.5, 0.5, 0.5],
    
    # for ["Susceptible", "Moderate", "Resistant"]
    p_opt = [7, 10, 14],
    rc_opt_par = [0.35, 0.25, 0.15],
    rrlex_par = [0.1, 0.01, .0001]
)


result = calculate_disease_severity(    
    data_path = "Original_Code/data/Data.csv",
    info_path = "Original_Code/data/Info.csv",
    add_year = True,
    
    number_applications_list = [0, 1, 2, 3],
    genetic_mechanistic_list = ["Susceptible", "Moderate", "Resistant"],
    
    required_para = required_para,
    
    path_result = None
)


result.to_csv(f"results.csv", index=None)

In [1]:
import pandas as pd

In [8]:
data = pd.read_csv("Data.csv")

for id, df in data.groupby("ID"):
    print(df["maximum_temperature"][(df["maximum_temperature"] >= 17) & (df["maximum_temperature"] <= 25)].count())
    break

88


In [1]:
# TEST

from typing import Dict, Optional, Tuple, Union, List

import numpy as np
import pandas as pd


def corp_parameters(
    crop_mechanistic: List[str] = None,
    crop_parameters_path: List[str] = None,
) -> Dict:
    
    para = ["ip_t_cof", "p_t_cof", "rc_t_input", "dvs_8_input", "rc_a_input", "fungicide", "fungicide_residual"]
      
    crop_para = dict()
    
    if (crop_parameters_path is not None) and (crop_mechanistic is not None):
        if len(crop_mechanistic) == len(crop_parameters_path):
            for i in range(len(crop_mechanistic)):
                crop_para[crop_mechanistic[i]] = dict()
                for p in para:
                    crop_para[crop_mechanistic[i]][p] = pd.read_excel(crop_parameters_path[i], engine="openpyxl", sheet_name=p, header=None)

    return crop_para

In [17]:
CROP_PARAMETERS = {
    "Corn" : {
        
        'ip_t_cof': pd.DataFrame(
            {
                0: [10.00, 13.00, 15.50, 17.00, 20.00, 26.00, 30.00, 35.00],
                1: [0.00, 0.14, 0.27, 0.82, 1.00, 0.92, 0.41, 0.00]
            }
        ),
        
        'p_t_cof': pd.DataFrame(
            {
                0: [15.00, 20.00, 25.00],
                1: [0.60, 0.81, 1.00]
            }
        ),
        
        'rc_t_input': pd.DataFrame(
            {
                0: [15.00, 20.00, 22.50, 24.00, 26.00, 30.00],
                1: [0.22, 1.00, 0.44, 0.43, 0.41, 0.22]
            }
        ),
        
        'dvs_8_input': pd.DataFrame(
            {
                0: [110.00, 200.00, 350.00, 475.00, 610.00, 740.00, 1135.00, 1660.00, 1925.00, 2320.00, 2700.00],
                1: [0.00, 1.00, 2.00, 3.00, 4.00, 5.00, 6.00, 8.00, 9.00, 10.00, 11.00]
            }
        ),
        
        'rc_a_input': pd.DataFrame(
            {
                0: [0.00, 1.00, 2.00, 3.00, 4.00, 5.00, 6.00, 7.00],
                1: [1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00]
            }
        ),
        
        'fungicide': pd.DataFrame(
            {
                0: [1, 2, 3],
                1: [45, 62, 79],
                2: [0.4, 0.4, 0.4]
            }
        ),
        
        'fungicide_residual': pd.DataFrame(
            {
                0: [0.00, 5.00, 10.00, 15.00, 20.00],
                1: [1.00, 0.80, 0.50, 0.25, 0.00]
            }
        )        
    },

    "Soy" : {
        
        'ip_t_cof': pd.DataFrame(
            {
                0: [5.00, 11.00, 17.00, 23.00, 29.00, 35.00],
                1: [0.00, 0.33, 0.60, 1.00, 1.00, 0.00]
            }
        ),
        
        'p_t_cof': pd.DataFrame(
            {
                0: [0.00, 4.00, 8.00, 12.00, 16.00, 20.00, 24.00, 28.00, 32.00, 36.00, 40.00],
                1: [0.00, 0.31, 0.39, 0.53, 0.82, 1.00, 1.00, 0.75, 0.60, 0.30, 0.00]
            }
        ),
        
        'rc_t_input': pd.DataFrame(
            {
                0: [10.00, 12.50, 15.00, 17.50, 20.00, 23.00, 25.00, 27.50, 30.00],
                1: [0.00, 0.57, 0.88, 1.00, 1.00, 0.86, 0.61, 0.41, 0.00]
            }
        ),
        
        'dvs_8_input': pd.DataFrame(
            {
                0: [137.00, 366.00, 595.00, 824.00, 1053.00, 1282.00, 1511.00, 1740.00],
                1: [0.00, 1.00, 2.00, 3.00, 4.00, 5.00, 6.00, 7.00]
            }
        ),
        
        'rc_a_input': pd.DataFrame(
            {
                0: [0.00, 1.00, 2.00, 3.00, 4.00, 5.00, 6.00, 7.00],
                1: [1.0000, 1.0000, 1.0000, 0.1000, 0.0001, 0.0001, 0.0001, 0.0001]
            }
        ),
        
        'fungicide': pd.DataFrame(
            {
                0: [1, 2, 3],
                1: [45, 62, 79],
                2: [0.4, 0.4, 0.4]
            }
        ),
        
        'fungicide_residual': pd.DataFrame(
            {
                0: [0.00, 5.00, 10.00, 15.00, 20.00],
                1: [1.00, 0.80, 0.50, 0.25, 0.00]
            }
        )        
    }  
}

In [ ]:
import plotly.express as px
